In [ ]:
# %run initialize/5_cross_shore_transport.ipynb

## First import some necessary packages

In [ ]:
import os
import pathlib

import colorcet as cc
import dask.dataframe as dd
import geopandas as gpd
import holoviews as hv
import hvplot.pandas  # noqa: API import
import hvplot.xarray  # noqa: API import
import ipyleaflet
import numpy as np
import pandas as pd
import panel as pn

from random import shuffle, uniform

import IPython
import ipywidgets as widgets
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from IPython.display import HTML, display
from ipywidgets import interact
from matplotlib.animation import FuncAnimation
from matplotlib.ticker import MultipleLocator

# (Cross-shore) sediment transport
Welcome to the notebook of week 5! This notebook covers chapter 6 and 7 from the book. The layout of this notebook is as follows:
* Modes of sediment transport (Chapter 6)
* Net transport for secondary flow (Chapter 6)
* Equillibrium states (Chapter 7)
* Beach states (Chapter 7)

Each section contains questions for you to practice with (cross-shore) sediment transport. Let's get started!

## Modes of sediment transport

## Net transport for secondary flow

## Equillirium states
This section considers equillibrium shoreline profiles (section 7.2 from the book). As you know, the shoreface is highly dynamic and can be subject to change on both short and long time scales. We often choose to model this as a dynamic equillibrium profile. Different formulations exist for the equillibrium profile, and most are derived at least somewhat empirically.

## Beach states
This section considers beach states (section 7.3 from the book). We have previously discussed equillibrium profiles of the upper shoreface. However, a beach is hardly in equillibrium. In this section, the focus is on the high variability of the upper shoreface, for instance following an episodic event.